# 第三题：实现带有拉普拉斯修正的朴素贝叶斯(选做）

实验内容：
1. 叙述拉普拉斯修正的作用
2. 给出使用的数据集
3. 给出实现的代码，要有详细的注释
4. 给出模型评价指标的结果

# 1.拉普拉斯修正的作用

**拉普拉斯修正的含义：**

在训练集中总共的分类数用 N 表示；di 属性可能的取值数用 Ni 表示，因此原来的先验概率 P(c) 的计算公式由：

$$P(c)=\frac{D_c}{D}$$

被拉普拉斯修正为：

$$P(c)=\frac{D_c +1}{D+N}$$

类的条件概率P(x|c) 的计算公式由：

$$ P(x_i |c)=\frac{D_{c,x_i}}{D_c} $$

被拉普拉斯修正为：

$$ P(x_i |c)=\frac{D_{c,x_i}+1}{D_c +Ni} $$

由上，拉普拉斯修正的作用是：
- **避免其他属性携带的信息被训练集中未出现的属性值抹去。**
- 在离散属性任务中，由于是直接将频率当成概率，所以若一个属性未出现，则计算出的概率始终0，为了避免这种错误，可以使用拉普拉斯修正使之平滑。

## 2.给出使用的数据集

In [1]:
import numpy as np
spambase = np.loadtxt('spambase/spambase.data', delimiter = ",")
spamx = spambase[:, :57]
spamy = spambase[:, 57]

In [2]:
# 讲spamx数据类型转换为float64
spamx_binary = (spamx != 0).astype('float64')

分割数据集：

In [3]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(spamx_binary, spamy, test_size = 0.4, random_state = 32)

## 3.带有拉普拉斯修正的朴素贝叶斯实现

In [ ]:
class Laplace:
    '''
    处理二值特征的高斯朴素贝叶斯（拉普拉斯平滑）
    '''
    def __init__(self):
        '''
        初始化四个字典
        self.label_mapping     类标记 与 下标(int)
        self.probability_of_y  类标记 与 先验概率(float)
        self.mean              类标记 与 均值(np.ndarray)
        self.var               类标记 与 方差(np.ndarray)
        '''
        self.label_mapping = dict()
        self.probability_of_y = dict()
        self.mean = dict()
        self.var = dict()
        
    def _clear(self):
        '''
        为了防止一个实例反复的调用fit方法，我们需要每次调用fit前，将之前学习到的参数删除掉
        '''
        self.label_mapping.clear()
        self.probability_of_y.clear()
        self.mean.clear()
        self.var.clear()
    
    def fit(self, trainX, trainY):
        '''
        这里，我们要根据trainY内的类标记，针对每类，计算这类的先验概率，以及这类训练样本每个特征的均值和方差

        Parameters
        ----------
            trainX: np.ndarray, 训练样本的特征, 维度：(样本数, 特征数)
        
            trainY: np.ndarray, 训练样本的标记, 维度：(样本数, )
        '''
        
        # 先调用_clear
        self._clear()
        
        # 获取类标记
        labels = np.unique(trainY)
        
        # 添加类标记与下标的映射关系
        self.label_mapping = {label: index for index, label in enumerate(labels)}
        
        # 遍历每个类
        for label in labels:
            
            # 取出为label这类的所有训练样本，存为 x
            x = trainX[trainY == label, :]
            
            # 计算先验概率，用 x 的样本个数除以训练样本总个数，存储到 self.probability_of_y 中，键为 label，值为先验概率
            # YOUR CODE HERE
            self.probability_of_y[label] = (x.shape[0]+1)/ (trainX.shape[0]+labels.shape[0])
            
        
    def predict(self, testX):
        '''
        给定测试样本，预测测试样本的类标记，这里我们要实现化简后的公式

        Parameters
        ----------
            testX: np.ndarray, 测试的特征, 维度：(测试样本数, 特征数)
    
        Returns
        ----------
            prediction: np.ndarray, 预测结果, 维度：(测试样本数, )
        '''
        
        # 初始化一个空矩阵 results，存储每个样本属于每个类的概率，维度是 (测试样本数，类别数)，每行表示一个样本，每列表示一个特征
        results = np.empty((testX.shape[0], len(self.probability_of_y)))
        
        # 初始化一个列表 labels，按 self.label_mapping 的映射关系存储所有的标记，一会儿会在下面的循环内部完成存储
        labels = [0] * len(self.probability_of_y)
        
        # 遍历当前的类，label为类标记，index为下标，我们将每个样本预测出来的这个 label 的概率，存到 results 中的第 index 列
        for label, index in self.label_mapping.items():
            
            # 先验概率存为 py
            py = self.probability_of_y[label]
            
            # 使用变换后的公式，计算所有特征的条件概率之和，存为sum_of_conditional_probability
            # YOUR CODE HERE
            # 这里 还没有 实现！！！
            sum_of_conditional_probability = - 0.5 * np.sum(np.power((testX - self.mean[label]),2) / self.var[label], axis = 1)- 0.5 * np.sum(np.log(2 * np.pi * self.var[label]))
                                            
            # debug
            assert sum_of_conditional_probability.shape == (len(testX), )
            
            # 使用变换后的公式，将 条件概率 与 log先验概率 相加，存为result，维度应该是 (测试样本数, )
            # YOUR CODE HERE
            result = sum_of_conditional_probability+np.log(py)
            
            # debug
            assert result.shape == (len(testX), )
            
            # 将所有测试样本属于当前这类的概率，存入到results中
            results[:, index] = result
            
            # 将当前的label，按index顺序放入到labels中
            labels[index] = label
        
        # 将labels转换为np.ndarray
        np_labels = np.array(labels)
        
        # 循环结束后，就计算出了给定测试样本，当前样本属于这类的概率的近似值，存放在了results中，每行对应一个样本，每列对应一个特征
        # 我们要求每行的最大值对应的下标，也就是求每个样本，概率值最大的那个下标是什么，结果存入max_prob_index中
        # YOUR CODE HERE
        max_prob_index = np.argmax(results, axis=1)
        
        # debug
        assert max_prob_index.shape == (len(testX), )
        
        # 现在得到了每个样本最大概率对应的下标，我们需要把这个下标变成 np_labels 中的标记
        # 使用上面小技巧中的第五点求解
        # YOUR CODE HERE
        prediction = np_labels[max_prob_index]
        
        # debug
        assert prediction.shape == (len(testX), )
        
        # 返回预测结果
        return prediction

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
model = Laplace()
model.fit(trainX, trainY)

prediction = model.predict(testX)

In [ ]:
acc = accuracy_score(testY, prediction)
pre = precision_score(testY, prediction)
rec = recall_score(testY, prediction)
f1 = f1_score(testY, prediction)